# 텍스트 마이닝을 활용한 영화 평점 감성 분석

## 1) 영화 평점 데이터 크롤링

In [66]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

base_url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=208077&target=after&page={}"
url = base_url.format(1)
res = requests.get(url)

soup = BeautifulSoup(res.text)
tds = soup.select('table.list_netizen > tbody > tr > td.title')

for td in tds:
    movie_title = td.select_one('a.movie').text.strip()
    score = td.select_one('div.list_netizen_score > em').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    print(movie_title, score, comment, sep=' :: ')
    print('-------------------------------------------')

스파이더맨: 노 웨이 홈 :: 4 :: ....망할 애새끼....
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 징짜 레전드 영화였음. 최고의 영화
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 8 :: 이제까지의 스파이더맨들을 하나로 엮는 스토리텔링과 마블 시리즈 중 가장 강렬한 성장통을 겪은 피터 파커의 굳건한 눈빛에 박수를...
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 재미있게봤습니다. 앞으로가기대
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 2 :: 노잼인데 다들 찬양하네차라리 쿠키가 재밌음
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 앞부분 웃으면서 보다가 중반부부터 눈물 줄줄 흘려가지고 마지막에는 눈물 주르르륵 흘려서 눈 아팠음...ㅠ
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 4 :: 나는 이전작을 안봐서 감흥이 별로 없었다... 그냥 덕후들 팬들만 부러울뿐 ㅜㅜ 해리포터도 힘내주라...??
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 소름 2번..첫번째/일단 내가 네이버 평점쓰면서 감상포인트 전부 선택한게 얼마만인지 모르겟다 . 두번째/토비 나올때
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 8 :: 모든걸 잃은 스파이더맨
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 인피니티 워 이후로 가장 슬픈 마블 영화네요 ㅠㅠ
---

In [67]:
import time
import random

comment_list=[] # 저장 할 리스트
for page in range(1, 1001):
    url = base_url.format(page)
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'lxml')
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    
    for td in tds:
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        comment_list.append((score, comment)) # 리스트에 추가
    delay = round(random.uniform(0.2, 1.2), 2) # 크롤링을 위해 페이지 전환을 빠르게 하게되면 네이버에서 막을 수 있기에 delay를 걸어준다.
    time.sleep(delay)
    

In [62]:
print(comment_list)

[('4', '....망할 애새끼....'), ('10', '징짜 레전드 영화였음. 최고의 영화'), ('8', '이제까지의 스파이더맨들을 하나로 엮는 스토리텔링과 마블 시리즈 중 가장 강렬한 성장통을 겪은 피터 파커의 굳건한 눈빛에 박수를...'), ('10', '재미있게봤습니다. 앞으로가기대'), ('2', '노잼인데 다들 찬양하네차라리 쿠키가 재밌음'), ('10', '앞부분 웃으면서 보다가 중반부부터 눈물 줄줄 흘려가지고 마지막에는 눈물 주르르륵 흘려서 눈 아팠음...ㅠ'), ('4', '나는 이전작을 안봐서 감흥이 별로 없었다... 그냥 덕후들 팬들만 부러울뿐 ㅜㅜ 해리포터도 힘내주라...??'), ('10', '소름 2번..첫번째/일단 내가 네이버 평점쓰면서 감상포인트 전부 선택한게 얼마만인지 모르겟다 . 두번째/토비 나올때'), ('8', '모든걸 잃은 스파이더맨'), ('10', '인피니티 워 이후로 가장 슬픈 마블 영화네요 ㅠㅠ'), ('4', '....망할 애새끼....'), ('10', '징짜 레전드 영화였음. 최고의 영화'), ('8', '이제까지의 스파이더맨들을 하나로 엮는 스토리텔링과 마블 시리즈 중 가장 강렬한 성장통을 겪은 피터 파커의 굳건한 눈빛에 박수를...'), ('10', '재미있게봤습니다. 앞으로가기대'), ('2', '노잼인데 다들 찬양하네차라리 쿠키가 재밌음'), ('10', '앞부분 웃으면서 보다가 중반부부터 눈물 줄줄 흘려가지고 마지막에는 눈물 주르르륵 흘려서 눈 아팠음...ㅠ'), ('4', '나는 이전작을 안봐서 감흥이 별로 없었다... 그냥 덕후들 팬들만 부러울뿐 ㅜㅜ 해리포터도 힘내주라...??'), ('10', '소름 2번..첫번째/일단 내가 네이버 평점쓰면서 감상포인트 전부 선택한게 얼마만인지 모르겟다 . 두번째/토비 나올때'), ('8', '모든걸 잃은 스파이더맨'), ('10', '인피니티 워 이후로 가장 슬픈 마블 영화네요 ㅠㅠ'), ('4', '....망할 애새끼....'), ('10', '징짜 레

In [46]:
import pandas as pd
df = pd.DataFrame(comment_list,columns=['rate','comment'])
df.to_csv('spider_naver_comment.csv', encoding='utf-8', index = False)

## 2) 텍스트 전처리를 위한 데이터 셋 살펴보기

In [37]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/nexussord/HUFS_MLP/main/spider_naver_comment.csv")

In [47]:
df.head()

,rate,comment
0,10,정말 재미있게봤어요.
1,10,스파이더맨의 새로운 세계관이 눈물나는 반가움을 선물해주네요. 철부지 좌충우돌 십대 ...
2,10,역대급이다.. 진심 마블 영화중에 인피니티 다음으로 마음을 울렸다.. 내가 어릴때 ...
3,10,
4,10,사랑해요 다정한 스파이더맨 ..


In [48]:
df.shape

(10000, 2)

In [49]:
df.isnull().sum() # 데이터 무결성 확인

rate       0
comment    0
dtype: int64

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   rate     10000 non-null  object
 1   comment  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [59]:
df['comment'][200]

'정말 재미있게봤어요.'

In [52]:
len(df['comment'].values.sum())

370020

## 3) 한국어 텍스트 전처리

In [53]:
!pip install konlpy==0.5.1 jpype1 

In [54]:
import re

def filter(text):
    korean = re.compile('[^ ㄱ-ㅣ가-힣]') # 한글만 추출할 수 있게끔
    result = korean.sub('', text)
    return result

In [ ]:
filter(df['text'][0])